In [1]:
import cv2
import numpy as np
import mediapipe as mp
import time

# **Keyboard Layout and States**


In [2]:
keys = [
    list("1234567890") + ['-', '=', 'del'],
    list("qwertyuiop") + ['[', ']'],
    ['caps'] + list("asdfghjkl") + [';', "'"],
    list("zxcvbnm") + [',', '.', '/', 'shift'],
    ['space']
]

special_chars = {
    '1': '!', '2': '@', '3': '#', '4': '$', '5': '%',
    '6': '^', '7': '&', '8': '*', '9': '(', '0': ')',
    '-': '_', '=': '+', '[': '{', ']': '}',
    ';': ':', "'": '"', ',': '<', '.': '>', '/': '?'
}

caps_lock_on = False
shift_on = False


# **Keyboard Drawing Function**


In [3]:
def draw_keyboard(img, key_list, hovered_key=None, pressed_key=None, caps_state=False, shift_state=False):
    img_width = img.shape[1]
    
    for i, row in enumerate(key_list):
        row_width = 400 if i == 4 else len(row) * 60 - 5
        row_start_x = (img_width - row_width) // 2
        
        for j, key in enumerate(row):
            if key == 'space':
                w, h = 400, 80
                x = (img_width - w) // 2
            elif key in ['del', 'caps', 'shift']:
                w, h = 70, 80
                x = row_start_x + j * 60
            else:
                w, h = 55, 80
                x = row_start_x + j * 60
            
            y = 120 + i * 85

            if key == pressed_key:
                color, border_color, text_color = (80,200,80), (40,160,40), (255,255,255)
            elif key == hovered_key:
                color, border_color, text_color = (100,150,255), (60,110,200), (255,255,255)
            elif key == 'caps' and caps_state:
                color, border_color, text_color = (255,200,100), (200,150,50), (0,0,0)
            elif key == 'shift' and shift_state:
                color, border_color, text_color = (255,150,200), (200,100,150), (0,0,0)
            else:
                color, border_color, text_color = (240,240,240), (100,100,100), (0,0,0)

            cv2.rectangle(img, (x, y), (x + w, y + h), color, cv2.FILLED)
            cv2.rectangle(img, (x, y), (x + w, y + h), border_color, 2)
            
            if key == 'del':
                text, font_scale = 'DEL', 0.6
            elif key == 'caps':
                text, font_scale = 'CAPS', 0.5
            elif key == 'shift':
                text, font_scale = 'SHIFT', 0.4
            elif key == 'space':
                text, font_scale = 'SPACE', 0.9
            else:
                if shift_state and key in special_chars:
                    text = special_chars[key]
                elif caps_state and key.isalpha():
                    text = key.upper()
                else:
                    text = key
                font_scale = 0.8 if len(text) > 1 else 1.0

            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)[0]
            text_x = x + (w - text_size[0]) // 2
            text_y = y + (h + text_size[1]) // 2
            cv2.putText(img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, 2)

    return img

# **Hover Detection**


In [4]:
def detect_hover_key(x_tip, y_tip, img_width):
    for i, row in enumerate(keys):
        row_width = 400 if i == 4 else len(row) * 60 - 5
        row_start_x = (img_width - row_width) // 2

        for j, key in enumerate(row):
            if key == 'space':
                w, h = 400, 80
                x = (img_width - w) // 2
            elif key in ['del', 'caps', 'shift']:
                w, h = 70, 80
                x = row_start_x + j * 60
            else:
                w, h = 55, 80
                x = row_start_x + j * 60

            y = 120 + i * 85
            padding = 10
            if (x - padding) < x_tip < (x + w + padding) and (y - padding) < y_tip < (y + h + padding):
                return key
    return None

# **Main Loop and Detection**


In [6]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

typed_text = ""
last_press_time = 0
pressed_key = None
press_feedback_time = 0

while True:
    success, img = cap.read()
    if not success:
        break
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    hover_key = None
    current_time = time.time()
    if current_time - press_feedback_time > 0.2:
        pressed_key = None

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            h, w, _ = img.shape
            lm = hand_landmarks.landmark
            lm_list = [(int(p.x * w), int(p.y * h)) for p in lm]

            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            index_tip = lm_list[8]
            thumb_tip = lm_list[4]

            cv2.circle(img, index_tip, 15, (255, 0, 255), -1)
            cv2.circle(img, index_tip, 15, (150, 0, 150), 3)
            cv2.circle(img, thumb_tip, 10, (255, 100, 0), -1)
            cv2.circle(img, thumb_tip, 10, (180, 70, 0), 3)

            hover_key = detect_hover_key(*index_tip, w)
            pinch_distance = np.linalg.norm(np.array(index_tip) - np.array(thumb_tip))

            if hover_key and pinch_distance < 50:
                if current_time - last_press_time > 1.0:
                    pressed_key = hover_key
                    last_press_time = current_time
                    press_feedback_time = current_time

                    if pressed_key == 'del':
                        typed_text = typed_text[:-1]
                    elif pressed_key == 'caps':
                        caps_lock_on = not caps_lock_on
                    elif pressed_key == 'shift':
                        shift_on = not shift_on
                    elif pressed_key == 'space':
                        typed_text += ' '
                    else:
                        char_to_add = pressed_key
                        if shift_on and pressed_key in special_chars:
                            char_to_add = special_chars[pressed_key]
                        elif pressed_key.isalpha():
                            char_to_add = pressed_key.upper() if (caps_lock_on or shift_on) else pressed_key.lower()
                        typed_text += char_to_add

    img = draw_keyboard(img, keys, hovered_key=hover_key, pressed_key=pressed_key, caps_state=caps_lock_on, shift_state=shift_on)

    cv2.rectangle(img, (50, 30), (1200, 100), (250, 250, 250), -1)
    cv2.rectangle(img, (50, 30), (1200, 100), (100, 100, 100), 2)

    display_text = typed_text
    if len(display_text) > 50:
        display_text = "..." + display_text[-47:]

    cv2.putText(img, display_text, (60, 75), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 2)
    cv2.putText(img, f"CAPS: {'ON' if caps_lock_on else 'OFF'} | SHIFT: {'ON' if shift_on else 'OFF'}",
                (50, 115), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
    cv2.putText(img, "Pinch: index+thumb | SHIFT: toggle button | CAPS: toggle case | Press 'q' to quit",
                (50, img.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    cv2.imshow("Virtual Keyboard", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()